In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from sklearn.datasets import fetch_20newsgroups 

newsgroup = fetch_20newsgroups()

# les messages
messages = newsgroup['data']

# répartition des messages
solution = pd.Series(newsgroup['target'])
Ydf = pd.DataFrame(solution.value_counts().sort_index(), columns=['msg_count'])
Ydf.insert(0, 'group_names', newsgroup['target_names'])

print(Ydf)
print(len(messages))

                 group_names  msg_count
0                alt.atheism        480
1              comp.graphics        584
2    comp.os.ms-windows.misc        591
3   comp.sys.ibm.pc.hardware        590
4      comp.sys.mac.hardware        578
5             comp.windows.x        593
6               misc.forsale        585
7                  rec.autos        594
8            rec.motorcycles        598
9         rec.sport.baseball        597
10          rec.sport.hockey        600
11                 sci.crypt        595
12           sci.electronics        591
13                   sci.med        594
14                 sci.space        593
15    soc.religion.christian        599
16        talk.politics.guns        546
17     talk.politics.mideast        564
18        talk.politics.misc        465
19        talk.religion.misc        377
11314


In [197]:
# fonction qui va séparer les données de chaque message
def format_message(str):

    lines = str.split('\n')

# 1. GET HEADERS
    headers = {}
    message_index = 0
    last_header = ''
    for i,l in enumerate(lines) :
        # Détetcter le premier saut de ligne qui represente le début du message
        if l == '' :
            message_index = i
            break
        
        head = l.split(':',1)
        # gestion des exceptions
        if len(head) < 2 :
            # aucune entête
            if last_header == '' :
                if 'TRASH' not in headers :  headers['TRASH'] = ''
                headers['TRASH'] += (head[0])
            # entête multi-ligne
            else : 
                headers[last_header] += (head[0])
        # cas normal
        else :
            last_header = head[0]
            headers[head[0]] = head[1][1:]

# 2. GET MESSAGE
    message = '\n'.join(lines[message_index:])

    return {'headers': headers, 'message': message}

In [198]:
for k,v in format_message(messages[690])['headers'].items():
    print(k,':',v)

TRASH :  cs.utexas.edu!uunet!olivea!sgigate!sgiblab!adagio.panasonic.com!nntp-server.caltech.edu!keith
Subject : Re: <Political Atheists?
From : keith@cco.caltech.edu (Keith Allan Schneider) <930401.112329.0u1.rusnews.w165w@mantis.co.uk> <11710@vice.ICO.TEK.COM>
Organization : California Institute of Technology, Pasadena
NNTP-Posting-Host : lloyd.caltech.edu
Lines : 17


In [199]:
from pprint import pprint

# Faire la liste de tous les headers pour voir haha
df_head = pd.DataFrame()
headers_list = {}

for i, msg in enumerate(messages):
    headers = format_message(msg)['headers']
    for header in headers.keys():
        if header not in headers_list:
            headers_list[header] = 1
        else:
            headers_list[header] += 1

print(len(headers_list))

# gardons seulement les entêtes qui sont présents au moins 10 fois
main_head = dict(filter(lambda a: True if a[1] > 10 else False, headers_list.items()))

print(len(main_head))


119
41
